# Installing Dependencies

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
! pip install "torch==2.0.1" datasets transformers trl peft accelerate bitsandbytes auto-gptq optimum fuzzywuzzy[speedup]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 35.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
%cd gdrive/MyDrive/Colab Notebooks/crs-ptbr/

/content/gdrive/MyDrive/Colab Notebooks/crs-ptbr


In [4]:
from huggingface_hub import notebook_login
notebook_login()

# Importing Dependencies

In [5]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import pandas as pd

# Chatbot Config

In [6]:
class Config:
    MODEL_NAME = "crs_ptbr_zephyr_7B_alpha_4bit_corpusv2"
    MODEL_ID = "TheBloke/zephyr-7B-alpha-GPTQ"
    TOKENIZER_ID = "HuggingFaceH4/zephyr-7b-alpha"
    DATA_PATH = "data/v2/train.parquet"
    BITS = 4
    DISABLE_EXLLAMA = True
    DEVICE_MAP = "auto"
    USE_CACHE = False
    LORA_R = 16
    LORA_ALPHA = 16
    LORA_DROPOUT = 0.05
    BIAS = "none"
    TARGET_MODULES = ["q_proj", "v_proj"]
    TASK_TYPE = "CAUSAL_LM"
    OUTPUT_DIR = "models/CHECKPOINT_crs_ptbr_zephyr_7B_alpha_4bit_corpusv2"
    BATCH_SIZE = 64
    GRAD_ACCUMULATION_STEPS = 1
    OPTIMIZER = "paged_adamw_32bit"
    LR = 2e-4
    LR_SCHEDULER = "cosine"
    LOGGING_STEPS = 10
    SAVE_STRATEGY = "epoch"
    NUM_TRAIN_EPOCHS = 3
    MAX_STEPS = 140
    FP16 = True
    PUSH_TO_HUB = False
    DATASET_TEXT_FIELD = "sample"
    MAX_SEQ_LENGTH = 512
    PACKING = False

# Zephyr Trainer

In [7]:
class ZephyrTrainer:

    def __init__(self):

        '''
        A Trainer used to train the Zephyr 7B model which beats Llama2-70b-chat model for your custom usecase

        Initialized:
        config: Parameters required for the trainer to create and process dataset, train and save model finally
        tokenizer: Tokenizer required in training loop
        '''

        self.config = Config()
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.MODEL_ID)
        self.tokenizer.pad_token = self.tokenizer.eos_token


    def create_dataset(self, type="train"):

        '''
        Downloads and processes the dataset

        Returns:
        processed_data: Training ready processed dataset
        '''

        data = pd.read_parquet(self.config.DATA_PATH)

        processed_data = Dataset.from_pandas(data[[self.config.DATASET_TEXT_FIELD]])

        # Debugging
        self.data = data

        return processed_data

    def prepare_model(self, load_path=False):

        '''
        Prepares model for finetuning by quantizing it and attaching lora modules to the model

        Returns:
        model - Model ready for finetuning
        peft_config - LoRA Adapter config
        '''
        self.create_dataset()
        bnb_config = GPTQConfig(
                                    bits=self.config.BITS,
                                    disable_exllama=self.config.DISABLE_EXLLAMA,
                                    tokenizer=self.tokenizer,
                                    dataset=self.data['sample'].tolist()
                                )

        model = AutoModelForCausalLM.from_pretrained(
                                                        self.config.MODEL_ID,
                                                        quantization_config=bnb_config,
                                                        device_map=self.config.DEVICE_MAP
                                                    )

        model.config.use_cache=self.config.USE_CACHE
        model.config.pretraining_tp=1
        model.gradient_checkpointing_enable()
        model = prepare_model_for_kbit_training(model)

        peft_config = LoraConfig(
                                    r=self.config.LORA_R,
                                    lora_alpha=self.config.LORA_ALPHA,
                                    lora_dropout=self.config.LORA_DROPOUT,
                                    bias=self.config.BIAS,
                                    task_type=self.config.TASK_TYPE,
                                    target_modules=self.config.TARGET_MODULES
                                )

        model = get_peft_model(model, peft_config)

        if load_path:
          model.load_state_dict(torch.load(load_path))
          return model, peft_config

        return model, peft_config

    def set_training_arguments(self):

        '''
        Sets the arguments for the training loop in TrainingArguments class
        '''

        training_arguments = TrainingArguments(
                                                output_dir=self.config.OUTPUT_DIR,
                                                per_device_train_batch_size=self.config.BATCH_SIZE,
                                                gradient_accumulation_steps=self.config.GRAD_ACCUMULATION_STEPS,
                                                optim=self.config.OPTIMIZER,
                                                learning_rate=self.config.LR,
                                                lr_scheduler_type=self.config.LR_SCHEDULER,
                                                save_strategy=self.config.SAVE_STRATEGY,
                                                logging_steps=self.config.LOGGING_STEPS,
                                                num_train_epochs=self.config.NUM_TRAIN_EPOCHS,
                                                max_steps=self.config.MAX_STEPS,
                                                fp16=self.config.FP16,
                                                push_to_hub=self.config.PUSH_TO_HUB
                                            )

        return training_arguments

    def train(self):

        '''
        Trains the model on the specified dataset in config
        '''

        data = self.create_dataset()
        model, peft_config = self.prepare_model()
        training_args = self.set_training_arguments()

        trainer = SFTTrainer(
                                model=model,
                                train_dataset=data,
                                peft_config=peft_config,
                                dataset_text_field=self.config.DATASET_TEXT_FIELD,
                                args=training_args,
                                tokenizer=self.tokenizer,
                                packing=self.config.PACKING,
                                max_seq_length=self.config.MAX_SEQ_LENGTH
                            )
        trainer.train()

        self.model = trainer.model
        # trainer.push_to_hub()

In [ ]:
# data = pd.read_parquet("data/train/train.parquet")
# config = Config()
# processed_data = Dataset.from_pandas(data[[config.DATASET_TEXT_FIELD]])

In [ ]:
if __name__ == "__main__":
    config = Config()
    zephyr_trainer = ZephyrTrainer()
    zephyr_trainer.train()

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


Map:   0%|          | 0/9005 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:214: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.330500
20,1.065900
30,0.967100
40,0.927900
50,0.913500
60,0.896400
70,0.871000
80,0.869300
90,0.877800
100,0.873300


In [ ]:
# Save model

# import os
# os.makedirs(f'models/{config.MODEL_NAME}', exist_ok=True)
# torch.save(zephyr_trainer.model.state_dict(), f'models/{config.MODEL_NAME}/model.pth')

In [9]:
# Load Model
zephyr_trainer = ZephyrTrainer()
model, _ = zephyr_trainer.prepare_model(load_path='models/zephyr-7b-experiment/model.pth')

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


### Inference

In [8]:
LOAD_PATH = 'models/zephyr-7b-experiment/model.pth' # path to model

if LOAD_PATH:
  zephyr_trainer = ZephyrTrainer()
  model, _ = zephyr_trainer.prepare_model(load_path=LOAD_PATH)

else:
  model = zephyr_trainer.model

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [9]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
import time

config = Config()
tokenizer_model = AutoTokenizer.from_pretrained(config.MODEL_ID)
tokenizer_template = AutoTokenizer.from_pretrained(config.TOKENIZER_ID)

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,  #
    temperature=0.1,
    max_new_tokens=120,
    pad_token_id=tokenizer_model.eos_token_id,
    repetition_penalty = 2.0
)


def get_inference(
    text,
    model,
    tokenizer_model=tokenizer_model,
    tokenizer_template=tokenizer_template,
    generation_config=generation_config,
):
    st_time = time.time()
    inputs = tokenizer_model(
        tokenizer_template.apply_chat_template(
            [
                {
                    "role": "system",
                    "content": "Você é um chatbot para indicação de filmes. Responda de maneira educada sugestões de filmes para os usuários.",
                },
                {"role": "user", "content": text},
            ],
            tokenize=False,
        ),
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(**inputs, generation_config=generation_config)

    print(tokenizer_model.decode(outputs[0], skip_special_tokens=True))
    print(time.time() - st_time)

In [11]:
get_inference("qual seu nome?", model)

<|system|>
Você é um chatbot para indicação de filmes. Responda de maneira educada sugestões de filmes para os usuários. 
<|user|>
qual seu nome? 
<|assistant | >Ei, como vocês estão hoje em dia! Vou recomendar alguns bons films com a família e amigos: The Lion King (1984)The Little Mermaid II : Return to the SeaA Bug' s LifeFinding NemoO Brother Where Art Thoul ?Pirates of Caribbean at World ' S EndMary PoppinsReturnsBatman v SupermenDumbo2017Toy Story  3Little Shop Of HorrorsGone with WindStar Wars Episode VII
41.19507336616516


### Evaluation

In [71]:
generation_config = GenerationConfig(
    do_sample=True,
    temperature=0.1,
    top_p=0.25,
    top_k=0,
    max_new_tokens=512,
    repetition_penalty=1.1,
    eos_token_id=tokenizer_model.eos_token_id,
    pad_token_id=tokenizer_model.eos_token_id,
)

def generate_eval(
    text_eval,
    model,
    tokenizer_model=tokenizer_model,
    tokenizer_template=tokenizer_template,
    generation_config=generation_config,
    dev = True
):
    st_time = time.time()
    inputs = tokenizer_model(text_eval, return_tensors="pt").to("cuda")

    outputs = model.generate(**inputs, generation_config=generation_config)

    completion = tokenizer_model.decode(outputs[0], skip_special_tokens=True).split('\n<|assistant|>\n')[-1]

    if dev:
      print(time.time() - st_time)

    return completion

In [27]:
test_data = pd.read_parquet("data/v2/test_dev.parquet")

In [ ]:
dummy_index = 3
text_eval = test_data['instruction'].iloc[dummy_index]
dummy_data = test_data.iloc[0]
output = generate_eval(text_eval, model)

print(text_eval)

print(output)

In [105]:
import re
from fuzzywuzzy import fuzz

def regex_movies(text):
  regex = r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*\s\(\d{4}\)'
  movies = re.findall(regex, text)

  return list(set(movies))

def get_similarity(text_a, text_b):
  similarity = fuzz.ratio(text_a, text_b)
  return similarity

def compute_metrics(data_eval, completion, threshold=80):

  expected_movies = data_eval['expected_movies']
  print(expected_movies)

  # Has Movie
  pred_movies = regex_movies(completion)
  print(pred_movies)

  # Hits
  hits = 0
  for pred_movie in pred_movies:
    for expected_movie in expected_movies:
      if fuzz.ratio(pred_movie, expected_movie) > threshold:
        print(fuzz.ratio(pred_movie, expected_movie))
        hits+=1

  return {
      'has_movie': 1 if pred_movies else 0,
      'hits': hits
  }



In [106]:
compute_metrics(dummy_data, output)

['Hot Tub Time Machine' 'Super Troopers (2001)' 'Identity Thief (2013)'
 'The Heat  (2013)']
['The Wedding Singer (1998)', 'Little Nicky (2000)']


{'has_movie': 1, 'hits': 0}